In [2]:
%matplotlib inline

import logging
import os
import numpy as np
import pandas as pd
import csv

In [3]:
DATA_ROOT = "data"
RESULTS_ROOT = "results"

In [4]:
from tqdm import tqdm
def read_csv(fname):
    df = pd.read_csv(fname)
    df["issued date"] = pd.to_datetime(df["issued date"], format='%Y-%m-%d')
    return df

file_path = os.path.join(DATA_ROOT, "text_data_medical_imaging.csv")
df = read_csv(file_path)

In [5]:
df.head()

,pn,issued date,abstract,claims,f_citation
0,10504621,2019-12-10,"According to one embodiment, a medical image p...",What is claimed is: \n\n 1. A medical image p...,0
1,10504252,2019-12-10,An imaging data processing apparatus comprises...,The invention claimed is: \n\n 1. An imaging ...,0
2,10504229,2019-12-10,A medical image processing apparatus according...,What is claimed is: \n\n 1. A medical image p...,0
3,10504227,2019-12-10,This disclosure generally pertains to methods ...,We claim: \n\n 1. A method for streamlining a...,0
4,10499883,2019-12-10,The systems and methods described herein relat...,What is claimed is: \n\n 1. A method for spat...,0


In [6]:
# pns = list(map(str, list(df["pn"])))
pns = df["pn"].astype(str).tolist()
print(pns[:10])
print("length:", len(pns))

['10504621', '10504252', '10504229', '10504227', '10499883', '10499868', '10497118', '10492764', '10492763', '10489910']
length: 1877


## Forward Citation 정보 추가

In [7]:
import sys
from tqdm import tqdm
from uspto.search import search, search_count
import uspto
import uspto.analysis.citation
import uspto.crawl, uspto.parse
import time
uspto.shared.set_html_dir("/share/uspto/html")

In [8]:
def get_f_citation_info(pn, timeout=1000):
    headers = {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36"
    }
    query = "ref/{}".format(pn)
    
    ref_pns = []
    page = 1
    total = search_count(query, timeout=timeout, headers=headers)
    pns50, is_next = search(query, timeout=timeout, headers=headers, page=page)
    ref_pns += pns50
    while is_next != None:
        page += 1
        pns50, is_next = uspto.search.search(query, is_next, timeout=timeout, headers=headers)
        ref_pns += pns50
        # print("Page {}: {} \n".format(page, str(pns50)))
        if page % 10 == 0:
            # print("################# Progress: {}%".format(((len(pns) + page * 50) / total) * 100))
            pns_json = json.dumps(pns)
            f = open("pns9070.txt", "w")
            f.write(pns_json)
            f.write("Page: {}".format(page))
            f.close()
    
    total_len = len(ref_pns)
    forward_df_list = []
    p = uspto.Patent(pn, use_db=False)
    issued_year = int(p.date.strftime('%Y-%m-%d')[:4])
    for idx, ref_pn in enumerate(ref_pns):
        if "D" in ref_pn: continue
        try:
            p = uspto.Patent(ref_pn, use_db=False)
            # print("pn: ", str(ref_pn), "(" + str(idx+1) + "/" + str(total_len) + ")")
            issued_date = p.date.strftime('%Y-%m-%d')[:4] if p.date != None else ''
            forward_df_list.append([ref_pn, issued_date])
        except KeyboardInterrupt:
            f.close()
            break
        except (FileNotFoundError, IndexError, ConnectionError): # TODO 제대로 크롤링 하지 못한 html이 그대로 남아 있는 경우가 존재.
            # print("\tpn: ", ref_pn, " not found or exist but something wrong. and try crwal pn =", ref_pn)
            if uspto.crawl.save_html(ref_pn, overwrite=True, retry=10):
                # print("\tCrawling success! and parse again..")
                time.sleep(8)
                try:
                    p = uspto.Patent(ref_pn, use_db=False)
                    # print("\tpn: ", ref_pn, "(" + str(idx+1) + "/" + str(total_len) + ")")
                    issued_date = p.date.strftime('%Y-%m-%d')[:4] if p.date != None else ''
                    forward_df_list.append([ref_pn, issued_date])
                except:
                    # print("\tUnexpected error:", sys.exc_info()[0])
                    # print("\tUnexpected error:", ref_pn)
                    continue

            else:
                # print("\tCrawling failed. Continue.:", ref_pn)
                continue
        except:
            # print("Unexpected error:", sys.exc_info()[0])
            # continue
            raise

    forward_df = pd.DataFrame(forward_df_list, columns=["pn", "year"])
    forward_df["year"] = pd.to_numeric(forward_df["year"])

    given_set = list(range(1984, 2021))
    temp = forward_df["year"].value_counts(sort=True)

    count_df = pd.Series(0, index=given_set)
    count_df = count_df.add(temp, fill_value=0)
    count_df.name = pn
    return count_df

In [11]:
from tqdm import tqdm
counts = pd.DataFrame(columns=list(range(1984, 2021)))

In [12]:
for pn in tqdm(pns):
    if pn in counts.index: continue
    p = uspto.Patent(pn, use_db=False)
    success = False
    while not success:
        try:
            count_df = get_f_citation_info(pn, timeout=20)
            success = True
            counts = counts.append(count_df)
        except KeyboardInterrupt:
            break
        except:
            print("retry!")
            continue


  1%|          | 10/1877 [00:25<1:04:06,  2.06s/it]

retry!
retry!



  1%|▏         | 24/1877 [01:54<57:13,  1.85s/it]  

retry!
retry!



  2%|▏         | 38/1877 [03:18<51:58,  1.70s/it]

...


 59%|█████▊    | 1100/1877 [1:34:59<27:39,  2.14s/it]

retry!
retry!



 59%|█████▉    | 1114/1877 [1:36:33<27:39,  2.17s/it]

retry!
retry!



100%|██████████| 1877/1877 [3:59:50<00:00,  9.57s/it]

In [13]:
counts

,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
10504621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10504252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10504229,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10504227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10499883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10499868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10497118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10492764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10492763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10489910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
counts.to_csv(os.path.join(DATA_ROOT, "forward_info.csv"), index_label="pn")

In [29]:
forward_info = pd.read_csv("forward_info.csv")
forward_info.head()

FileNotFoundError: File b'forward_info.csv' does not exist

In [21]:
df.head()

,pn,issued date,abstract,claims,f_citation
0,10504621,2019-12-10,"According to one embodiment, a medical image p...",What is claimed is: \n\n 1. A medical image p...,0
1,10504252,2019-12-10,An imaging data processing apparatus comprises...,The invention claimed is: \n\n 1. An imaging ...,0
2,10504229,2019-12-10,A medical image processing apparatus according...,What is claimed is: \n\n 1. A medical image p...,0
3,10504227,2019-12-10,This disclosure generally pertains to methods ...,We claim: \n\n 1. A method for streamlining a...,0
4,10499883,2019-12-10,The systems and methods described herein relat...,What is claimed is: \n\n 1. A method for spat...,0


In [24]:
final_df = pd.merge(df, forward_info, on="pn")
final_df.head()

,pn,issued date,abstract,claims,f_citation,1984,1985,1986,1987,1988,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,10504621,2019-12-10,"According to one embodiment, a medical image p...",What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10504252,2019-12-10,An imaging data processing apparatus comprises...,The invention claimed is: \n\n 1. An imaging ...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10504229,2019-12-10,A medical image processing apparatus according...,What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,10504227,2019-12-10,This disclosure generally pertains to methods ...,We claim: \n\n 1. A method for streamlining a...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10499883,2019-12-10,The systems and methods described herein relat...,What is claimed is: \n\n 1. A method for spat...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
final_df.set_index("pn")
final_df.head()

,pn,issued date,abstract,claims,f_citation,1984,1985,1986,1987,1988,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,10504621,2019-12-10,"According to one embodiment, a medical image p...",What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10504252,2019-12-10,An imaging data processing apparatus comprises...,The invention claimed is: \n\n 1. An imaging ...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10504229,2019-12-10,A medical image processing apparatus according...,What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,10504227,2019-12-10,This disclosure generally pertains to methods ...,We claim: \n\n 1. A method for streamlining a...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10499883,2019-12-10,The systems and methods described herein relat...,What is claimed is: \n\n 1. A method for spat...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
final_df.to_csv(os.path.join(DATA_ROOT, "data2.csv"))

In [47]:
final = pd.read_csv(os.path.join(DATA_ROOT, "data.csv"))
# del final["Unnamed: 0"]

In [48]:
final

,Unnamed: 0,pn,issued date,abstract,claims,f_citation,1984,1985,1986,1987,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,10504621,2019-12-10,"According to one embodiment, a medical image p...",What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,10504252,2019-12-10,An imaging data processing apparatus comprises...,The invention claimed is: \n\n 1. An imaging ...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,10504229,2019-12-10,A medical image processing apparatus according...,What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,10504227,2019-12-10,This disclosure generally pertains to methods ...,We claim: \n\n 1. A method for streamlining a...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,10499883,2019-12-10,The systems and methods described herein relat...,What is claimed is: \n\n 1. A method for spat...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,10499868,2019-12-10,"According to one embodiment, an X-ray diagnost...",What is claimed is: \n\n 1. An X-ray diagnost...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,10497118,2019-12-03,A user had to manually select a cross-sectiona...,What is claimed is: \n\n 1. An image processi...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,7,10492764,2019-12-03,"According to one embodiment, an ultrasound dia...",What is claimed is: \n\n 1. An ultrasound dia...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,10492763,2019-12-03,The present invention provides improved medica...,What is claimed: \n\n 1. An ultrasound imagin...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9,10489910,2019-11-26,A medical image processing apparatus according...,What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Class 정보 추가 (Class of the patent and the references)

In [143]:
import collections
def get_class_info(pn, log=False):
    p = uspto.Patent(pn)
    ref_us = p.ref_us
    p_cpcs = list(set(["{} {}".format(pc.level1, pc.level2) for pc in p.class_cpc]))
    ref_cpcs = []
    for ref in ref_us:
        ref_pn = ref.pn
        p_ref = None
        if "/" in ref_pn:
            if log: print("SKIP: {}".format(ref_pn))
            continue
        try:
            p_ref = uspto.Patent(ref_pn, use_db=False)
            if log: print("pn: {} was aleady existed".format(ref_pn))
        except KeyboardInterrupt: break
        except (FileNotFoundError, IndexError): # TODO 제대로 크롤링 하지 못한 html이 그대로 남아 있는 경우가 존재.
            if log: print("pn: {} not found or exist but something wrong. and try crwal pn = {}".format(ref_pn, ref_pn))
            if uspto.crawl.save_html(ref_pn, overwrite=True, retry=3):
                if log: print("\tCrawling success! and parse again..")
                time.sleep(8)
                try:
                    p_ref = uspto.Patent(ref_pn, use_db=False)
                except:
                    if log: print("\tUnexpected error:", sys.exc_info()[0])
                    continue
            else:
                if log: print("\tCrawling failed. Continue.")
                continue
        except:
            if log: print("Unexpected error:", sys.exc_info()[0])
            continue
        
        if p_ref: ref_cpcs += ["{} {}".format(c.level1, c.level2) for c in p_ref.class_cpc]
    
    ref_cpcs_count = collections.Counter(ref_cpcs)
    ref_cpcs_unique = list(set(ref_cpcs))
    return p_cpcs, ref_cpcs, ref_cpcs_unique, ref_cpcs_count

In [144]:
import collections
pn = "4430749"
p_cpcs, ref_cpcs, ref_cpcs_unique, ref_cpcs_count = get_class_info(pn)
# print("ref_us:", ref_us)
print("p_cpcs:", p_cpcs)
print("ref_cpcs:", ref_cpcs)
print("ref_cpcs_count:", ref_cpcs_count)
print("ref_cpcs_unique:", ref_cpcs_unique)

p_cpcs: ['G06T 1', 'G06T 5', 'H04N 5']
ref_cpcs: ['H04N 5', 'G21K 4', 'A61B 6', 'G06T 11', 'Y10S 378', 'A61B 6', 'G06T 11', 'Y10S 378', 'H04N 5', 'G06T 5', 'H04N 5', 'G06T 5', 'A61B 6', 'A61B 6', 'A61B 6', 'H05G 1', 'A61B 6', 'H04N 5', 'A61B 6']
ref_cpcs_count: Counter({'A61B 6': 7, 'H04N 5': 4, 'G06T 11': 2, 'Y10S 378': 2, 'G06T 5': 2, 'G21K 4': 1, 'H05G 1': 1})
ref_cpcs_unique: ['G06T 11', 'A61B 6', 'Y10S 378', 'G21K 4', 'H05G 1', 'H04N 5', 'G06T 5']


In [64]:
final.head()

,Unnamed: 0,pn,issued date,abstract,claims,f_citation,1984,1985,1986,1987,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,10504621,2019-12-10,"According to one embodiment, a medical image p...",What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,10504252,2019-12-10,An imaging data processing apparatus comprises...,The invention claimed is: \n\n 1. An imaging ...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,10504229,2019-12-10,A medical image processing apparatus according...,What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,10504227,2019-12-10,This disclosure generally pertains to methods ...,We claim: \n\n 1. A method for streamlining a...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,10499883,2019-12-10,The systems and methods described herein relat...,What is claimed is: \n\n 1. A method for spat...,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [145]:
pns = list(final["pn"])
pns

[10504621,
 10504252,
 10504229,
 10504227,
 10499883,
 10499868,
 10497118,
 10492764,
 10492763,
 10489910,
 10489907,
 10485493,
 10484594,
 10482606,
 10482603,
 10481234,
 10478130,
 10478073,
 10478071,
 10475184,
 10475182,
 10470825,
 10468134,
 10467798,
 10467377,
 10463871,
 10463328,
 10463327,
 10460423,
 10460079,
 10459627,
 10456106,
 10456099,
 10456093,
 10456041,
 10455201,
 10453570,
 10453182,
 10452813,
 10452812,
 10448910,
 10445891,
 10445886,
 10445462,
 10444960,
 10442767,
 10438380,
 10438368,
 10438349,
 10438348,
 10436915,
 10433804,
 10433728,
 10430550,
 10429457,
 10426444,
 10424237,
 10417804,
 10417778,
 10417759,
 10417517,
 10417326,
 10416833,
 10413270,
 10413253,
 10413252,
 10413236,
 10413186,
 10412377,
 10410341,
 10409235,
 10405831,
 10402969,
 10401975,
 10398395,
 10398382,
 10398286,
 10395420,
 10395364,
 10390785,
 10390728,
 10388037,
 10387612,
 10387196,
 10383602,
 10383593,
 10380718,
 10379692,
 10371933,
 10371499,
 10368836,

In [146]:
class_df = pd.DataFrame(columns=["pn", "p_cpcs", "ref_cpcs", "ref_cpcs_unique", "ref_cpcs_count"])
class_df

,pn,p_cpcs,ref_cpcs,ref_cpcs_unique,ref_cpcs_count


In [147]:
class_df = pd.DataFrame(columns=["pn", "p_cpcs", "ref_cpcs", "ref_cpcs_unique", "ref_cpcs_count"])
for pn in tqdm(pns):
    row = [pn] + list(get_class_info(str(pn)))
    class_df = class_df.append(pd.Series(row, index=class_df.columns), ignore_index=True)

class_df


100%|██████████| 1877/1877 [30:13<00:00,  3.85it/s]

,pn,p_cpcs,ref_cpcs,ref_cpcs_unique,ref_cpcs_count
0,10504621,"[G06T 11, G06T 7, G06T 2210, G16H 30, G06T 2207]","[G06F 19, H04N 1, G06T 3, H04N 19, H04N 21, H0...","[G06F 19, G06T 11, G06T 3, G06T 7, A61B 6, H04...","{'G06F 19': 1, 'H04N 1': 1, 'G06T 3': 1, 'H04N..."
1,10504252,"[G06T 11, G06T 7, A61B 6, G06T 3, G06T 2207]","[A61B 6, G06T 7, G06T 2207, G06T 19, G06T 2219...","[G06T 7, A61B 6, A61B 8, G06F 3, G06T 2210, G0...","{'A61B 6': 2, 'G06T 7': 13, 'G06T 2207': 16, '..."
2,10504229,"[G06T 7, G06T 2207, A61B 2576, A61B 5]","[A61M 25, A61M 25, A61B 2090, A61B 5, A61N 1, ...","[A61B 2090, G06T 7, G01R 33, A61B 6, A61M 25, ...","{'A61M 25': 2, 'A61B 2090': 1, 'A61B 5': 5, 'A..."
3,10504227,"[G06T 7, G06T 2207, G06N 3, G06N 20]",[],[],{}
4,10499883,[A61B 8],"[A61B 8, G01S 15, G01S 15, G01S 15, G01S 7, G0...","[G01S 7, A61B 8, G01S 15]","{'A61B 8': 12, 'G01S 15': 4, 'G01S 7': 2}"
5,10499868,[A61B 6],[],[],{}
6,10497118,"[G06F 19, G06T 7, G16H 15, G16H 30, G06T 2207,...","[A61B 6, A61B 6, Y10S 378, G06F 17, G06F 17, G...","[G06F 19, G06T 7, A61B 6, Y10S 378, G06T 2207,...","{'A61B 6': 2, 'Y10S 378': 1, 'G06F 17': 3, 'G0..."
7,10492764,"[G06T 11, A61B 8, G06T 2210]",[],[],{}
8,10492763,"[G01S 7, A61B 8, G01S 15]","[G06T 5, Y10S 378, G06T 5, G06T 5, G06T 5, G06...","[G06T 11, G06T 7, Y10S 378, A61B 6, A61B 8, G0...","{'G06T 5': 11, 'Y10S 378': 1, 'G06T 2207': 14,..."
9,10489910,"[G06T 7, G06T 2207]","[A61B 5, A61B 5, A61B 5, G06T 7, G01R 33, G06T...","[G06T 7, G06K 9, G01R 33, A61B 8, A61N 5, A61N...","{'A61B 5': 7, 'G06T 7': 18, 'G01R 33': 7, 'G06..."


In [101]:
type(class_df[class_df['pn']==10504621]['p_cpcs'].item())

list

In [149]:
del class_df['ref_cpcs_count']

In [150]:
class_df.to_csv(os.path.join(DATA_ROOT, "class_info.csv"), index=False)

In [151]:
class_df[class_df["pn"]==10504621]["p_cpcs"].item()

['G06T 11', 'G06T 7', 'G06T 2210', 'G16H 30', 'G06T 2207']

In [84]:
final.to_csv(os.path.join(DATA_ROOT, "data.csv"))

In [154]:
final = pd.read_csv(os.path.join(DATA_ROOT, "data.csv"))
del final['p_cpcs']
del final['ref_cpcs']
del final['ref_cpcs_unique']

In [155]:
final.head()

,pn,issued date,abstract,claims,f_citation,1984,1985,1986,1987,1988,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,10504621,2019-12-10,"According to one embodiment, a medical image p...",What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10504252,2019-12-10,An imaging data processing apparatus comprises...,The invention claimed is: \n\n 1. An imaging ...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10504229,2019-12-10,A medical image processing apparatus according...,What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,10504227,2019-12-10,This disclosure generally pertains to methods ...,We claim: \n\n 1. A method for streamlining a...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10499883,2019-12-10,The systems and methods described herein relat...,What is claimed is: \n\n 1. A method for spat...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [156]:
final = pd.merge(final, class_df, on='pn')

In [157]:
final.head()

,pn,issued date,abstract,claims,f_citation,1984,1985,1986,1987,1988,...,2014,2015,2016,2017,2018,2019,2020,p_cpcs,ref_cpcs,ref_cpcs_unique
0,10504621,2019-12-10,"According to one embodiment, a medical image p...",What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[G06T 11, G06T 7, G06T 2210, G16H 30, G06T 2207]","[G06F 19, H04N 1, G06T 3, H04N 19, H04N 21, H0...","[G06F 19, G06T 11, G06T 3, G06T 7, A61B 6, H04..."
1,10504252,2019-12-10,An imaging data processing apparatus comprises...,The invention claimed is: \n\n 1. An imaging ...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[G06T 11, G06T 7, A61B 6, G06T 3, G06T 2207]","[A61B 6, G06T 7, G06T 2207, G06T 19, G06T 2219...","[G06T 7, A61B 6, A61B 8, G06F 3, G06T 2210, G0..."
2,10504229,2019-12-10,A medical image processing apparatus according...,What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"[G06T 7, G06T 2207, A61B 2576, A61B 5]","[A61M 25, A61M 25, A61B 2090, A61B 5, A61N 1, ...","[A61B 2090, G06T 7, G01R 33, A61B 6, A61M 25, ..."
3,10504227,2019-12-10,This disclosure generally pertains to methods ...,We claim: \n\n 1. A method for streamlining a...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[G06T 7, G06T 2207, G06N 3, G06N 20]",[],[]
4,10499883,2019-12-10,The systems and methods described herein relat...,What is claimed is: \n\n 1. A method for spat...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[A61B 8],"[A61B 8, G01S 15, G01S 15, G01S 15, G01S 7, G0...","[G01S 7, A61B 8, G01S 15]"


In [158]:
final.to_csv(os.path.join(DATA_ROOT, "data.csv"), index=False)

In [159]:
final = pd.read_csv(os.path.join(DATA_ROOT, "data.csv"))
final.head()

,pn,issued date,abstract,claims,f_citation,1984,1985,1986,1987,1988,...,2014,2015,2016,2017,2018,2019,2020,p_cpcs,ref_cpcs,ref_cpcs_unique
0,10504621,2019-12-10,"According to one embodiment, a medical image p...",What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"['G06T 11', 'G06T 7', 'G06T 2210', 'G16H 30', ...","['G06F 19', 'H04N 1', 'G06T 3', 'H04N 19', 'H0...","['G06F 19', 'G06T 11', 'G06T 3', 'G06T 7', 'A6..."
1,10504252,2019-12-10,An imaging data processing apparatus comprises...,The invention claimed is: \n\n 1. An imaging ...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"['G06T 11', 'G06T 7', 'A61B 6', 'G06T 3', 'G06...","['A61B 6', 'G06T 7', 'G06T 2207', 'G06T 19', '...","['G06T 7', 'A61B 6', 'A61B 8', 'G06F 3', 'G06T..."
2,10504229,2019-12-10,A medical image processing apparatus according...,What is claimed is: \n\n 1. A medical image p...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"['G06T 7', 'G06T 2207', 'A61B 2576', 'A61B 5']","['A61M 25', 'A61M 25', 'A61B 2090', 'A61B 5', ...","['A61B 2090', 'G06T 7', 'G01R 33', 'A61B 6', '..."
3,10504227,2019-12-10,This disclosure generally pertains to methods ...,We claim: \n\n 1. A method for streamlining a...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"['G06T 7', 'G06T 2207', 'G06N 3', 'G06N 20']",[],[]
4,10499883,2019-12-10,The systems and methods described herein relat...,What is claimed is: \n\n 1. A method for spat...,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,['A61B 8'],"['A61B 8', 'G01S 15', 'G01S 15', 'G01S 15', 'G...","['G01S 7', 'A61B 8', 'G01S 15']"


In [96]:
import csv
f = open(os.path.join(DATA_ROOT, "data.csv"), "r")
csv_reader = csv.reader(f)

In [98]:
for row in csv_reader:
    print(row)
    break

['10504621', '2019-12-10', 'According to one embodiment, a medical image processing apparatus includes processing circuitry. The processing circuitry is configured to extract a first medical image relating to a region of interest from a second medical image. The processing circuitry is configured to add, to the second medical image, reconstruction matrix information added to the first medical image.', 'What is claimed is: \n\n 1.  A medical image processing apparatus comprising processing circuitry configured to: extract a first medical image relating to a region of interest from a second medical\nimage;  and add, to the first medical image, reconstruction matrix information added to the second medical image.\n\n\n 2.  The apparatus according to claim 1, wherein the processing circuitry is further configured to display the reconstruction matrix information together with the first medical image.\n\n\n 3.  The apparatus according to claim 2, wherein the processing circuitry further displ